# **DocuPy**  
### _"Automate Documentation, Comments, and Unit Tests for Python Code"_  

## Overview  
DocuPy is a Gradio-powered tool designed to automate essential but time-consuming Python development tasks. It streamlines documentation, unit testing, and Python-to-C++ code conversion with AI-driven assistance.  

### Key Features  
✅ **Auto-Generate Docstrings & Comments** – Instantly improve code clarity and maintainability.  
✅ **Unit Test Generation** – Ensure reliability with AI-generated test cases.  
✅ **Python to C++ Conversion** – Seamlessly translate Python code to C++ with execution support.  

With an intuitive tab-based UI, DocuPy enhances productivity for developers of all levels. Whether you're documenting functions, validating code with tests, or exploring C++ conversions, this tool lets you focus on coding while it handles the rest.  

🔗 **Check out the repo**: [GitHub Repo](https://github.com/emads22/DocuPy)  

💡 **Have insights, feedback, or ideas?** Feel free to reach out. 

[<img src="https://img.shields.io/badge/GitHub-Emad-blue?logo=github" width="150">](https://github.com/emads22)


If you're running this notebook on **`Google Colab`**, ensure you install the required libraries by running the following command:

```bash
!pip install -q openai anthropic python-dotenv gradio huggingface_hub transformers
```
Otherwise, make sure to activate the Conda environment `docupy` that already includes these modules:

```bash
conda activate docupy
```

In [ ]:
# Uncomment the following command when running on Google Colab
# !pip install -q openai anthropic python-dotenv gradio huggingface_hub transformers 

## Setup and Install Dependencies

- Start by installing all necessary libraries.

In [ ]:
# imports
import os
import io
import sys
import subprocess
import openai
import anthropic
import google.generativeai as google_genai
import gradio as gr
from openai import OpenAI
# from google.colab import userdata
from dotenv import load_dotenv
from pathlib import Path
from huggingface_hub import login, InferenceClient
from transformers import AutoTokenizer

## Add Secrets to the Colab Notebook

- Add the API keys for OpenAI, Claude, and Gemini to authenticate and access their respective models and services.


In [ ]:
# # Log in to Hugging Face using the token and add it to git credentials
# hf_token = userdata.get('HF_TOKEN')
# login(token=hf_token, add_to_git_credential=True)

# # Endpoint URL for accessing the Code Qwen model through Hugging Face
# CODE_QWEN_URL = userdata.get('CODE_QWEN_URL')

# # Initialize inference clients with every model using API keys
# gpt = openai.OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
# claude = anthropic.Anthropic(api_key=userdata.get('ANTHROPIC_API_KEY'))
# google_genai.configure(api_key=userdata.get('GOOGLE_API_KEY'))
# code_qwen = InferenceClient(CODE_QWEN_URL, token=hf_token)

## Alternatively, if not running on Google Colab, Load Environment Variables for API Keys

- Use the `load_dotenv()` function to securely load API keys from a `.env` file.
- Ensure that the `.env` file is located in the same directory as your script or Jupyter Notebook.
- The `.env` file should include the required API keys for OpenAI, Claude, and Gemini.

In [ ]:
load_dotenv()

# Log in to Hugging Face using the token and add it to git credentials
hf_token = os.getenv('HF_TOKEN')
login(token=hf_token, add_to_git_credential=True)

# Endpoint URL for accessing the Code Qwen model through Hugging Face
CODE_QWEN_URL = os.getenv('CODE_QWEN_URL')

# Initialize inference clients with every model using API keys
gpt = openai.OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
claude = anthropic.Anthropic(api_key=os.getenv('ANTHROPIC_API_KEY'))
google_genai.configure(api_key=os.getenv('GOOGLE_API_KEY'))
code_qwen = InferenceClient(CODE_QWEN_URL, token=hf_token)

## Define Required Constants

- Initialize the essential constants required for the application's functionality.
- Configure the system and user prompts specific to each task or feature.


In [ ]:
# Models
OPENAI_MODEL = "gpt-4o"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
GEMINI_MODEL = "gemini-1.5-pro"
CODE_QWEN_MODEL = "Qwen/CodeQwen1.5-7B-Chat"

MODELS_IN_USE = ["GPT", "Claude", "Gemini", "CodeQwen"]

MAX_TOKENS = 2000

ACTION_A = "commenting"
ACTION_B = "testing"
ACTION_C = "converting"

# Define and create the path for the "temp_files" directory within the current script's directory
TEMP_DIR = Path.cwd() / "temp_files"
TEMP_DIR.mkdir(parents=True, exist_ok=True)

PYTHON_SCRIPT_EASY = """
import time

def reverse_string(s):
    return s[::-1]

if __name__ == "__main__":
    start_time = time.time()
    text = "Hello, World!"
    print(f"- Original string: {text}")
    print("- Reversed string:", reverse_string(text))
    execution_time = time.time() - start_time  
    print(f"\\n=> Execution Time: {execution_time:.6f} seconds")
"""

PYTHON_SCRIPT_INTERMEDIATE = """
import time

def is_palindrome(s):
    s = s.lower().replace(" ", "")  
    return s == s[::-1]

if __name__ == "__main__":
    start_time = time.time()
    text = "Racecar"
    if is_palindrome(text):
        print(f"- '{text}' is a palindrome!")
    else:
        print(f"- '{text}' is Not a palindrome.")
    execution_time = time.time() - start_time  
    print(f"\\n=> Execution Time: {execution_time:.6f} seconds")
"""

PYTHON_SCRIPT_HARD = """
import time

def generate_primes(limit):
    primes = []
    for num in range(2, limit + 1):
        if all(num % p != 0 for p in primes):
            primes.append(num)
    return primes

if __name__ == "__main__":
    start_time = time.time()
    n = 20
    print(f"- Generating primes up to: {n}")
    print("- Prime numbers:", generate_primes(n))
    execution_time = time.time() - start_time  
    print(f"\\n=> Execution Time: {execution_time:.6f} seconds")
"""

PYTHON_SCRIPTS = {
    "reverse_string" : PYTHON_SCRIPT_EASY,
    "is_palindrome" : PYTHON_SCRIPT_INTERMEDIATE,
    "generate_primes" : PYTHON_SCRIPT_HARD,
    "custom" : """
# Write your custom Python script here
if __name__ == "__main__":
    print("Hello, World!")
"""
}

# Relative system prompts
SYSTEM_PROMPT_COMMENTS = """
You are an AI model specializing in enhancing Python code documentation.
Generate detailed and precise docstrings and inline comments for the provided Python code.
Ensure the docstrings clearly describe the purpose, parameters, and return values of each function.
Inline comments should explain complex or non-obvious code segments.
Do not include any introductions, explanations, conclusions, or additional context.
Return only the updated Python code enclosed within ```python ... ``` for proper formatting and syntax highlighting.
"""

SYSTEM_PROMPT_TESTS = """
You are an AI model specializing in generating comprehensive unit tests for Python code.
Create Python unit tests that thoroughly validate the functionality of the given code.
Use the `unittest` framework and ensure edge cases and error conditions are tested.
Do not include any comments, introductions, explanations, conclusions, or additional context.
Return only the unit test code enclosed within ```python ... ``` for proper formatting and syntax highlighting.
"""

SYSTEM_PROMPT_CONVERT = """
You are an AI model specializing in high-performance code translation.
Translate the given Python code into equivalent, optimized C++ code.
Focus on:
- Using efficient data structures and algorithms.
- Avoiding unnecessary memory allocations and computational overhead.
- Ensuring minimal risk of integer overflow by using appropriate data types.
- Leveraging the C++ Standard Library (e.g., `<vector>`, `<algorithm>`) for performance and readability.
Produce concise and efficient C++ code that matches the functionality of the original Python code.
Do not include any comments, introductions, explanations, conclusions, or additional context..
Return only the C++ code enclosed within ```cpp ... ``` for proper formatting and syntax highlighting.
"""

In [ ]:
# Relative user prompts
def user_prompt_comments(python_code):
    user_prompt = f"""
Add detailed docstrings and inline comments to the following Python code:

```python
{python_code}
```
"""
    return user_prompt

def user_prompt_tests(python_code):
    user_prompt = f"""
Generate unit tests for the following Python code using the `unittest` framework:

```python
{python_code}
```
"""
    return user_prompt

def user_prompt_convert(python_code):
    user_prompt = f"""
Convert the following Python code into C++:

```python
{python_code}
``` 
"""
    return user_prompt

### Define the Tab Functions

- Develop dedicated functions for each service: documenting Python code, generating unit tests, and converting Python to C++.
- Structure each function to handle user input, process it using the selected AI model, and display the generated output seamlessly.
- Ensure the functionality of each tab aligns with its specific purpose, providing an intuitive and efficient user experience.


In [ ]:
def stream_gpt(system_prompt, user_prompt):
    stream = gpt.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=True)
    reply = ""
    for chunk in stream:
        reply += chunk.choices[0].delta.content or ""
        yield reply.replace("```python\n", "").replace("```cpp\n", "").replace("```", "")

def stream_claude(system_prompt, user_prompt):
    response = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=MAX_TOKENS,
        system=system_prompt,
        messages=[{"role": "user", "content": user_prompt}],
    )
    reply = ""
    with response as stream:
        for text in stream.text_stream:
            reply += text
            yield reply.replace("```python\n", "").replace("```cpp\n", "").replace("```", "")

def stream_gemini(system_prompt, user_prompt):
    gemini = google_genai.GenerativeModel(
        model_name=GEMINI_MODEL,
        system_instruction=system_prompt
    )
    stream = gemini.generate_content(
        contents=user_prompt,
        stream=True
    )
    reply = ""
    for chunk in stream:
        reply += chunk.text or ""
        yield reply.replace("```python\n", "").replace("```cpp\n", "").replace("```", "")

def stream_code_qwen(system_prompt, user_prompt):
    tokenizer = AutoTokenizer.from_pretrained(CODE_QWEN_MODEL)
    model_input = tokenizer.apply_chat_template(
        conversation=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        tokenize=False,
        add_generation_prompt=True
    )
    stream = code_qwen.text_generation(
        prompt=model_input,
        stream=True,
        details=True,
        max_new_tokens=MAX_TOKENS
    )
    reply = ""
    for chunk in stream:
        reply += chunk.token.text or ""
        yield reply.replace("```python\n", "").replace("```cpp\n", "").replace("```", "")

def set_prompts(user_input, action):
    action = action.lower()

    if action == ACTION_A.lower():
        system_prompt = SYSTEM_PROMPT_COMMENTS
        user_prompt = user_prompt_comments(user_input)
    elif action == ACTION_B.lower():
        system_prompt = SYSTEM_PROMPT_TESTS
        user_prompt = user_prompt_tests(user_input)
    elif action == ACTION_C.lower():
        system_prompt = SYSTEM_PROMPT_CONVERT
        user_prompt = user_prompt_convert(user_input)
    else:
        return None, None
    
    return system_prompt, user_prompt

def stream_response(user_input, model, action):
    system_prompt, user_prompt = set_prompts(user_input, action)
    if not all((system_prompt, user_prompt)):
        raise ValueError("Unknown Action")

    match model:
        case "GPT":
            yield from stream_gpt(system_prompt, user_prompt)

        case "Claude":
            yield from stream_claude(system_prompt, user_prompt)

        case "Gemini":
            yield from stream_gemini(system_prompt, user_prompt)

        case "CodeQwen":
            yield from stream_code_qwen(system_prompt, user_prompt)
                
def generate_comments(python_code, selected_model):
    for model in MODELS_IN_USE:
        if model == selected_model:
            yield from stream_response(python_code, model, action=ACTION_A)
            return  # Exit the function immediately after exhausting the generator
    raise ValueError("Unknown Model")

def generate_tests(python_code, selected_model):
    for model in MODELS_IN_USE:
        if model == selected_model:
            yield from stream_response(python_code, model, action=ACTION_B)
            return  # Exit the function immediately after exhausting the generator
    raise ValueError("Unknown Model")

def convert_code(python_code, selected_model):
    for model in MODELS_IN_USE:
        if model == selected_model:
            yield from stream_response(python_code, model, action=ACTION_C)
            return  # Exit the function immediately after exhausting the generator
    raise ValueError("Unknown Model")

## Running Code Functions

- Functions that dynamically execute Python or C++ code provided as a string and captures its output.
- This is useful for evaluating Python or C++ code snippets and returning their results programmatically.

### IMPORTANT WARNING:
The functions that dynamically execute Python or C++ code provided as input.
While powerful, this is extremely dangerous if the input code is not trusted.
Any malicious code can be executed, including:
  - Deleting files or directories
  - Stealing sensitive data (e.g., accessing environment variables or credentials)
  - Running arbitrary commands that compromise the system

Sharing this notebook with this code snippet can allow attackers to exploit this functionality 
by passing harmful code as input. 

If you share this notebook or use this function:
  1. Only accept input from trusted sources.
  2. Consider running the code in a sandboxed environment (e.g., virtual machine or container).
  3. Avoid using this function in publicly accessible applications or notebooks without strict validation.

In [ ]:
def run_python_exec(code):
    try:
        # Capture stdout using StringIO
        output = io.StringIO()

        # Redirect stdout to StringIO
        sys.stdout = output

        # Execute the provided Python code
        exec(code)
    finally:
        # Restore original stdout
        sys.stdout = sys.__stdout__

    # Return the captured output
    return output.getvalue()

# Improved running python function
def run_python(code):
    # Save the Python code to a file
    with open(TEMP_DIR / "python_code.py", "w") as python_file:
        python_file.write(code)

    try:
        # Execute the Python code
        result = subprocess.run(
            ["python", str(TEMP_DIR / "python_code.py")],
            check=True, text=True, capture_output=True
        )

        # Return the program's output
        return result.stdout

    except subprocess.CalledProcessError as e:
        # Handle compilation or execution errors
        return f"An error occurred during execution:\n{e.stderr}"

    finally:
        # Clean up: Delete the Python code file and executable
        file_path = TEMP_DIR / "python_code.py"
        if file_path.exists():
            file_path.unlink()

def run_cpp(code):
    # Save the C++ code to a file
    with open(TEMP_DIR / "cpp_code.cpp", "w") as cpp_file:
        cpp_file.write(code)

    try:
        # Compile the C++ code
        subprocess.run(
            ["g++", "-o", str(TEMP_DIR / "cpp_code"), str(TEMP_DIR / "cpp_code.cpp")],
            check=True, text=True, capture_output=True
        )

        # Execute the compiled program
        result = subprocess.run(
            [str(TEMP_DIR / "cpp_code")],
            check=True, text=True, capture_output=True
        )

        # Return the program's output
        return result.stdout

    except subprocess.CalledProcessError as e:
        # Handle compilation or execution errors
        error_context = "during compilation" if "cpp_code.cpp" in e.stderr else "during execution"
        return f"An error occurred {error_context}:\n{e.stderr}"

    finally:
        # Clean up: Delete the C++ source file and executable
        for filename in ["cpp_code.cpp", "cpp_code", "cpp_code.exe"]:
            file_path = TEMP_DIR / filename
            if file_path.exists():
                file_path.unlink()

## Develop a User-Friendly Interface with Gradio

- Design a clean, intuitive, and user-centric interface using Gradio.
- Ensure responsiveness and accessibility to provide a seamless and efficient user experience.
- Focus on simplicity while maintaining functionality to cater to diverse user needs.



In [ ]:
# CSS styles for customizing the appearance of the Gradio UI elements.
css = """
.python { 
    background-color: #377ef0; 
    color: #ffffff; 
    padding: 0.5em; 
    border-radius: 5px; /* Slightly rounded corners */
}
.cpp { 
    background-color: #00549e; 
    color: #ffffff; 
    padding: 0.5em; 
    border-radius: 5px; 
}
.model { 
    background-color: #17a2b8; /* Vibrant cyan color */
    color: white; 
    font-size: 1.2em; 
    padding: 0.5em; 
    border: none; 
    border-radius: 5px; 
    cursor: pointer; 
}
.button { 
    height: 4em; 
    font-size: 1.5em; 
    padding: 0.5em 1em; 
    background-color: #e67e22; /* Vibrant orange */
    color: white; 
    border: none; 
    border-radius: 5px; 
    cursor: pointer; 
}
.run-button { 
    height: 3em; 
    font-size: 1.5em; 
    padding: 0.5em 1em; 
    background-color: #16a085; /* Rich teal color */
    color: white; 
    border: none; 
    border-radius: 5px; 
    cursor: pointer; 
}
.button:hover, .run-button:hover {
    background-color: #2c3e50; /* Dark navy for hover effect */
    color: #fff; 
}
"""

In [ ]:
# Tab to Document Code with Docstrings and Comments
def docs_comments_ui():
    with gr.Tab("Docstrings & Comments"):
        gr.Markdown("""
        ## Document Code with Docstrings and Comments
        This tab allows you to automatically generate docstrings and inline comments for your Python code.
        - Paste your Python code into the **`Python Code`** textbox.
        - Select your preferred model (GPT, Claude, Gemini, or CodeQwen) to process the code.
        - Click the **`Add Docstrings & Comments`** button to generate well-documented Python code.
        The generated code will appear in the **`Python Code with Docstrings and Comments`** textarea.
        """)
        with gr.Row():
            python = gr.Textbox(label="Python Code:", lines=20, value=PYTHON_SCRIPTS["custom"], elem_classes=["python"])
            python_with_comments = gr.TextArea(label="Python Code with Docstrings and Comments:", interactive=True, lines=20, elem_classes=["python"])
        with gr.Row():
            python_script = gr.Dropdown(choices=list(PYTHON_SCRIPTS.keys()), label="Select a Python script", value="custom", elem_classes=["model"])
            comments_btn = gr.Button("Add Docstrings & Comments", elem_classes=["button"])
            model = gr.Dropdown(["GPT", "Claude", "Gemini", "CodeQwen"], label="Select Model", value="GPT", elem_classes=["model"])
            
        python_script.change(
            fn=lambda script: PYTHON_SCRIPTS[script],
            inputs=[python_script],
            outputs=[python]
        )
        
        comments_btn.click(
            fn=lambda: "",
            inputs=None,
            outputs=[python_with_comments]
        ).then(
            fn=generate_comments,
            inputs=[python, model],
            outputs=[python_with_comments]
        )

        return python_with_comments

In [ ]:
# Tab to Generate Comprehensive Unit Tests
def unit_tests_ui():
    with gr.Tab("Unit Tests"):
        gr.Markdown("""
        ## Generate Comprehensive Unit Tests
        This tab helps you create unit tests for your Python code automatically.
        - Paste your Python code into the **`Python Code`** textbox.
        - Choose a model (GPT, Claude, Gemini, or CodeQwen) to generate the unit tests.
        - Click the **`Generate Unit Tests`** button, and the generated unit tests will appear in the **`Python Code with Unit Tests`** textarea.
        Use these unit tests to ensure your code behaves as expected.
        """)
        with gr.Row():
            python = gr.Textbox(label="Python Code:", lines=20, value=PYTHON_SCRIPTS["custom"], elem_classes=["python"])
            python_unit_tests = gr.TextArea(label="Python Code with Unit Tests:", interactive=True, lines=20, elem_classes=["python"])
        with gr.Row():
            python_script = gr.Dropdown(choices=list(PYTHON_SCRIPTS.keys()), label="Select a Python script", value="custom", elem_classes=["model"])
            unit_tests_btn = gr.Button("Generate Unit Tests", elem_classes=["button"])
            model = gr.Dropdown(["GPT", "Claude", "Gemini", "CodeQwen"], label="Select Model", value="GPT", elem_classes=["model"])
            
        python_script.change(
            fn=lambda script: PYTHON_SCRIPTS[script],
            inputs=[python_script],
            outputs=[python]
        )
        
        unit_tests_btn.click(
            fn=lambda: "",
            inputs=None,
            outputs=[python_unit_tests]
        ).then(
            fn=generate_tests,
            inputs=[python, model],
            outputs=[python_unit_tests]
        )

        return python_unit_tests

In [ ]:
# Tab to Convert Python Code to C++
def python_to_cpp_ui():
    with gr.Tab("Python to C++"):
        gr.Markdown("""
        ## Convert Python Code to C++
        This tab facilitates the conversion of Python code into C++.
        - Paste your Python code into the **`Python Code`** textbox.
        - Select your preferred model (GPT, Claude, Gemini, or CodeQwen) to perform the conversion.
        - Click **`Convert to C++`** to see the equivalent C++ code in the **`C++ Code`** textbox.
        Additional Features:
        - You can execute the Python or C++ code directly using the respective **`Run Python`** or **`Run C++`** buttons.
        - The output will appear in the respective result text areas below.
        """)
        with gr.Row():
            python = gr.Textbox(label="Python Code:", lines=20, value=PYTHON_SCRIPTS["custom"], elem_classes=["python"])
            cpp = gr.Textbox(label="C++ Code:", interactive=True, lines=20, elem_classes=["cpp"])
        with gr.Row():
            python_script = gr.Dropdown(choices=list(PYTHON_SCRIPTS.keys()), label="Select a Python script", value="custom", elem_classes=["model"])
            convert_btn = gr.Button("Convert to C++", elem_classes=["button"])
            model = gr.Dropdown(["GPT", "Claude", "Gemini", "CodeQwen"], label="Select Model", value="GPT", elem_classes=["model"])
        with gr.Row():
            run_python_btn = gr.Button("Run Python", elem_classes=["run-button"])
            run_cpp_btn = gr.Button("Run C++", elem_classes=["run-button"])
        with gr.Row():
            python_out = gr.TextArea(label="Python Result:", lines=10, elem_classes=["python"])
            cpp_out = gr.TextArea(label="C++ Result:", lines=10, elem_classes=["cpp"])

        python_script.change(
            fn=lambda script: PYTHON_SCRIPTS[script],
            inputs=[python_script],
            outputs=[python]
        )
        
        convert_btn.click(
            fn=lambda: "",
            inputs=None,
            outputs=[cpp]
        ).then(
            fn=convert_code,
            inputs=[python, model],
            outputs=[cpp]
        )
        run_python_btn.click(run_python, inputs=[python], outputs=[python_out])
        run_cpp_btn.click(run_cpp, inputs=[cpp], outputs=[cpp_out])

        return cpp, python_out, cpp_out

In [ ]:
# Combine the tabs into the main UI and handle tab switching
with gr.Blocks(css=css) as main_ui:
    with gr.Tabs() as tabs:
        comments_output = docs_comments_ui()
        tests_output = unit_tests_ui()
        cpp_output, python_out, cpp_out = python_to_cpp_ui()

    # Reset outputs on tab switch
    tabs.select(
        fn=lambda: ["", "", "", "", ""],
        inputs=None,
        outputs=[comments_output, 
                 tests_output, 
                 cpp_output, python_out, cpp_out]
    )

# Launch the app
main_ui.launch(inbrowser=True)